In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [14]:
nickname = []
rating = []
title= []
voice=[]
author=[]
review = []

In [15]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

- 지현) 소설문학 http://m.podbbang.com/audiobooks?category_id=100
- 미리) 오디오북스 http://m.podbbang.com/audiobooks?category_id=111
- 미리) 자기계발 http://m.podbbang.com/audiobooks?category_id=101
- 현진) 경제경영 http://m.podbbang.com/audiobooks?category_id=103
- 현진) 인문교양 http://m.podbbang.com/audiobooks?category_id=104
- 연주) 역사 http://m.podbbang.com/audiobooks?category_id=105
- 연주) 어린이동화 http://m.podbbang.com/audiobooks?category_id=106


In [16]:
driver = webdriver.Chrome('C:/Users/Lg/Desktop/데잇걸즈/파이썬/chromedriver') #각자 chromedriver가 설치되어있는 주소를 입력해주세요
driver.implicitly_wait(3)
driver.get("http://m.podbbang.com/audiobooks?category_id=101") #각자 맡은 카테고리 페이지 url을 입력해주세요
scroll(driver, 0.6) #스크롤 끝까지 내려
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [30]:
for i in range(1,len(soup.select('ul .audiobook-list-component__list>li .audiobook-list-item-component>h3 .title'))):
    element = driver.find_element_by_xpath(f'//*[@id="audiobookIndex"]/div[2]/div/ul/li[{i}]')
    driver.execute_script("arguments[0].click();", element)
    scroll(driver, 0.6)
    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')
    n = int(soup2.select('div.audiobook-reviews-component>h3.title>span')[0].text.split(' ')[1].strip('명')) #n정의
    for i in range((n-3)//12+1):
        더보기 = driver.find_element_by_xpath('//*[@id="audiobookDetails"]/div[5]/div/button[1]')
        더보기.click()
    time.sleep(1)
    html = driver.page_source
    soup3 = BeautifulSoup(html, 'html.parser') #댓글 다 나와있는 soup
    for i in range(n):
        c=soup3.select('dt.title')[0].text
        a=soup3.select('h3.nickname')[i].text
        b=soup3.select('div.star_rating>span')[i].get('style').split(' ')[1][:-2]
        d=soup3.select('dl.audiobook-details__info>dd')[1].text[3:]
        e=soup3.select('dl.audiobook-details__info>dd')[2].text[2:]
        f=soup3.select('li.audiobook-review-list-item>p.message')[i].text
        nickname.append(a)
        rating.append(b)
        title.append(c)
        voice.append(d)
        author.append(e)
        review.append(f)
    driver.back() # 책 목록으로 돌아가
    scroll(driver, 0.6) #스크롤 끝까지 내려

In [31]:
booklist_selfhelp = pd.DataFrame({
    'title': title,
    'nickname':nickname,
    'rating':rating,
    'voice': voice,
    'author': author,
    'review': review
    
})

booklist_selfhelp

,title,nickname,rating,voice,author,review


In [33]:
booklist_selfhelp = pd.read_csv('booklist_selfhelp.csv')

In [34]:
booklist_selfhelp

,Unnamed: 0,title,nickname,rating,voice,author,review
0,0,나는 왠지 대박날 것만 같아!,파진찬어른,100,손정현,손정현,필모보니깐 후덜덜하네요
1,1,나는 왠지 대박날 것만 같아!,greathogoo,100,손정현,손정현,재미있네요 ^^
2,2,나는 왠지 대박날 것만 같아!,짱구인생,100,손정현,손정현,잘 들었어요^^ 재밌네요
3,3,나는 왠지 대박날 것만 같아!,성석동달인손세차,100,손정현,손정현,잘들었습니다 ^^
4,4,나는 왠지 대박날 것만 같아!,시즈루,100,손정현,손정현,저자가 후배지망작가들에 대한 애정이 느껴지네요
...,...,...,...,...,...,...,...
720,720,일 잘하는 사람은 단순하게 합니다,촛불은영원해야한다,100,"전해리,이승준",박소연,감사합니다
721,721,일 잘하는 사람은 단순하게 합니다,gj주,100,"전해리,이승준",박소연,좋아요
722,722,일 잘하는 사람은 단순하게 합니다,우유속당근,100,"전해리,이승준",박소연,좋은 내용인거 같네요
723,723,일 잘하는 사람은 단순하게 합니다,듣고있어요,100,"전해리,이승준",박소연,도움됐어요
